# Conversational Journal Generation with Nudging

This notebook extends the synthetic journal generation with a two-way conversational nudging system.
When an entry is vague or potentially rich with unexplored tension, the system responds with a brief nudge that invites elaboration.

**Design goal**: Nudges should feel like natural curiosity from a thoughtful companion, not interrogation or therapy.

In [1]:
import asyncio
import json
import os
import random
import re
import sys
import yaml
import polars as pl

from dataclasses import dataclass, field
from datetime import datetime, timedelta
from pathlib import Path
from dotenv import load_dotenv
from openai import AsyncOpenAI
from pydantic import BaseModel, Field
from typing import Literal

# Add project root to path for prompts module
PROJECT_ROOT = (
    Path(__file__).parent.parent if "__file__" in dir() else Path.cwd().parent
)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Load environment variables
load_dotenv()

# Check for API Key
if not os.getenv("OPENAI_API_KEY"):
    print("WARNING: OPENAI_API_KEY not found in environment variables.")

In [2]:
# Configuration Loading
CONFIG_PATH = Path("config/synthetic_data.yaml")
if not CONFIG_PATH.exists():
    CONFIG_PATH = Path("../config/synthetic_data.yaml")

SCHWARTZ_VALUES_PATH = Path("config/schwartz_values.yaml")
if not SCHWARTZ_VALUES_PATH.exists():
    SCHWARTZ_VALUES_PATH = Path("../config/schwartz_values.yaml")


def load_config(path: str | Path) -> dict:
    with open(path, "r") as f:
        return yaml.safe_load(f)


config = load_config(CONFIG_PATH)
schwartz_config = load_config(SCHWARTZ_VALUES_PATH)

print("Configs loaded successfully.")
print(f"Available Persona Attributes: {list(config['personas'].keys())}")
print(f"Schwartz Values with elaborations: {list(schwartz_config['values'].keys())}")
print(f"Nudge config loaded: {list(config['nudge'].keys())}")

Configs loaded successfully.
Available Persona Attributes: ['age_ranges', 'cultures', 'professions', 'schwartz_values']
Schwartz Values with elaborations: ['Self-Direction', 'Stimulation', 'Hedonism', 'Achievement', 'Power', 'Security', 'Conformity', 'Tradition', 'Benevolence', 'Universalism']
Nudge config loaded: ['response_probability', 'response_modes', 'min_words', 'max_words']


## Data Models

Extended models for conversational journaling with nudges.

In [ ]:
# Base models (from journal_gen.ipynb)
class Persona(BaseModel):
    name: str = Field(description="Full name of the persona")
    age: str
    profession: str
    culture: str
    core_values: list[str] = Field(description="Top 3 Schwartz values")
    bio: str = Field(
        description="A short paragraph describing their background, stressors, and goals"
    )


class JournalEntry(BaseModel):
    """LLM-generated journal entry. Metadata (tone, verbosity, etc.) tracked separately."""

    date: str
    content: str


# Nudge models and schemas imported from src/
from src.models.nudge import NudgeCategory, NudgeResult, JournalTurn
from src.nudge.schemas import (
    NUDGE_DECISION_RESPONSE_FORMAT,
    NUDGE_RESPONSE_FORMAT,
    NUDGE_RESPONSE_RESPONSE_FORMAT,
)


class ConversationalEntry(BaseModel):
    """Complete conversational exchange for one journaling session."""

    initial_entry: JournalEntry
    nudge: NudgeResult | None = None
    response: JournalTurn | None = None  # User's response to the nudge
    # Metadata
    tone: str
    verbosity: str
    reflection_mode: str
    response_mode: str | None = None  # How the persona responded to the nudge


# JSON schemas for OpenAI structured output (non-nudge only)
PERSONA_SCHEMA = {
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "string"},
        "profession": {"type": "string"},
        "culture": {"type": "string"},
        "core_values": {"type": "array", "items": {"type": "string"}},
        "bio": {"type": "string"},
    },
    "required": ["name", "age", "profession", "culture", "core_values", "bio"],
}

JOURNAL_ENTRY_SCHEMA = {
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "date": {"type": "string"},
        "content": {"type": "string"},
    },
    "required": ["date", "content"],
}

PERSONA_RESPONSE_FORMAT = {
    "type": "json_schema",
    "name": "Persona",
    "schema": PERSONA_SCHEMA,
    "strict": True,
}

JOURNAL_ENTRY_RESPONSE_FORMAT = {
    "type": "json_schema",
    "name": "JournalEntry",
    "schema": JOURNAL_ENTRY_SCHEMA,
    "strict": True,
}

In [4]:
def build_value_context(values: list[str], schwartz_config: dict) -> str:
    """Build rich context about Schwartz values for persona generation.

    Args:
        values: List of Schwartz value names (e.g., ["Achievement", "Benevolence"])
        schwartz_config: The loaded schwartz_values.yaml config

    Returns:
        Formatted string with value elaborations for prompt injection
    """
    context_parts = []

    for value_name in values:
        if value_name not in schwartz_config["values"]:
            continue

        v = schwartz_config["values"][value_name]

        # Build a focused context block for this value
        context_parts.append(f"""
### {value_name}
**Core Motivation:** {v["core_motivation"].strip()}

**How this manifests in behavior:**
{chr(10).join(f"- {b}" for b in v["behavioral_manifestations"][:5])}

**Life domain expressions:**
- Work: {v["life_domain_expressions"]["work"].strip()}
- Relationships: {v["life_domain_expressions"]["relationships"].strip()}

**Typical stressors for this person:**
{chr(10).join(f"- {s}" for s in v["typical_stressors"][:4])}

**Typical goals:**
{chr(10).join(f"- {g}" for g in v["typical_goals"][:3])}

**Internal conflicts they may experience:**
{v["internal_conflicts"].strip()}

**Narrative guidance:**
{v["persona_narrative_guidance"].strip()}
""")

    return "\n".join(context_parts)

## Prompt Templates

In [5]:
# Prompt templates are stored in prompts/ folder as YAML files
# See prompts/__init__.py for the loader utility
from prompts import (
    persona_generation_prompt,
    journal_entry_prompt,
    nudge_decision_prompt,
    nudge_generation_prompt,
    nudge_response_prompt,
)

## LLM Client Setup

Using `gpt-5-mini`. 

**Note:** GPT-5 models do not support `temperature` or `top_p` parameters. Instead, use the `reasoning` parameter to control how much the model "thinks" before responding.

In [6]:
client = AsyncOpenAI()
MODEL_NAME = "gpt-5-mini-2025-08-07"
# MODEL_NAME = "gpt-5-nano-2025-08-07"

# Type alias for reasoning effort levels
ReasoningEffort = Literal["minimal", "low", "medium", "high"]

# Default reasoning effort - change this to affect all generations
DEFAULT_REASONING_EFFORT: ReasoningEffort = "high"


async def generate_completion(
    prompt: str,
    response_format: dict | None = None,
) -> str | None:
    """Generate a completion using the OpenAI Responses API (async).

    Uses DEFAULT_REASONING_EFFORT to control how much the model "thinks".
    Valid reasoning effort values: "minimal", "low", "medium", "high".
    """
    try:
        kwargs = {
            "model": MODEL_NAME,
            "input": [{"role": "user", "content": prompt}],
            "reasoning": {"effort": DEFAULT_REASONING_EFFORT},
        }

        if response_format:
            kwargs["text"] = {"format": response_format}

        response = await client.responses.create(**kwargs)
        return response.output_text

    except Exception as e:
        print(f"Error generating completion: {e}")
        return None

## Utility Functions

In [7]:
def _verbosity_targets(verbosity: str) -> tuple[int, int, int]:
    """Returns (min_words, max_words, max_paragraphs) as guidance for the LLM."""
    normalized = verbosity.strip().lower()
    if normalized.startswith("short"):
        return 25, 80, 1
    if normalized.startswith("medium"):
        return 90, 180, 2
    return 160, 260, 3


def _build_banned_pattern(banned_terms: list[str]) -> re.Pattern:
    """Build regex pattern to detect banned Schwartz value terms."""
    escaped = [re.escape(term) for term in banned_terms if term.strip()]
    if not escaped:
        return re.compile(r"$^")
    return re.compile(r"(?i)\b(" + "|".join(escaped) + r")\b")


def generate_date_sequence(
    start_date: str, num_entries: int, min_days: int = 2, max_days: int = 10
) -> list[str]:
    """Generate a sequence of dates with random intervals.

    Args:
        start_date: Starting date in YYYY-MM-DD format
        num_entries: Number of dates to generate
        min_days: Minimum days between entries
        max_days: Maximum days between entries

    Returns:
        List of date strings in YYYY-MM-DD format
    """
    dates = []
    current = datetime.strptime(start_date, "%Y-%m-%d")

    for i in range(num_entries):
        dates.append(current.strftime("%Y-%m-%d"))
        if i < num_entries - 1:
            days_gap = random.randint(min_days, max_days)
            current += timedelta(days=days_gap)

    return dates


# Banned terms include Schwartz value labels AND derivative adjectives
SCHWARTZ_BANNED_TERMS = [
    # Value labels
    "Self-Direction",
    "Stimulation",
    "Hedonism",
    "Achievement",
    "Power",
    "Security",
    "Conformity",
    "Tradition",
    "Benevolence",
    "Universalism",
    # Derivative adjectives and related terms
    "self-directed",
    "autonomous",
    "stimulating",
    "excited",
    "hedonistic",
    "hedonist",
    "pleasure-seeking",
    "achievement-oriented",
    "ambitious",
    "powerful",
    "authoritative",
    "secure",
    "conformist",
    "conforming",
    "traditional",
    "traditionalist",
    "benevolent",
    "kind-hearted",
    "universalistic",
    "altruistic",
    # Meta terms
    "Schwartz",
    "values",
    "core values",
]

BANNED_PATTERN = _build_banned_pattern(SCHWARTZ_BANNED_TERMS)

## Nudge Decision Logic

LLM-based classification to decide whether to nudge and which category.

The LLM analyzes entry content semantically to detect:
- **clarification** — Entry too vague to understand
- **elaboration** — Solid entry with unexplored depth  
- **tension_surfacing** — Hints at unresolved conflict
- **no_nudge** — Entry is complete and grounded

In [ ]:
from src.nudge import should_suppress_nudge, decide_nudge, format_previous_entries


# Test the LLM-based decision logic using the extracted module
async def test_nudge_decision():
    test_entry = JournalEntry(date="2024-01-15", content="Feeling off today.")
    should, category, reason = await decide_nudge(
        entry_content=test_entry.content,
        entry_date=test_entry.date,
        previous_entries=None,
        llm_complete=generate_completion,
    )
    print(
        f"Test vague entry: should_nudge={should}, category={category}, reason={reason}"
    )

    test_entry2 = JournalEntry(
        date="2024-01-15",
        content="Had a meeting with the team about the project deadline. It was fine, I guess. We sorted out the schedule.",
    )
    should2, category2, reason2 = await decide_nudge(
        entry_content=test_entry2.content,
        entry_date=test_entry2.date,
        previous_entries=None,
        llm_complete=generate_completion,
    )
    print(
        f"Test hedging entry: should_nudge={should2}, category={category2}, reason={reason2}"
    )


await test_nudge_decision()

## Nudge Generation

In [ ]:
from src.nudge import generate_nudge_text

## Nudge Response Generation

In [ ]:
from src.nudge import select_response_mode, generate_nudge_response

## Conversational Pipeline

In [ ]:
@dataclass
class ConversationalPipelineResult:
    """Complete results from one persona's conversational generation pipeline."""

    persona_id: int
    persona: Persona | None
    entries: list[ConversationalEntry]
    persona_prompt: str
    entry_prompts: list[str]
    nudge_prompts: list[str] = field(default_factory=list)
    response_prompts: list[str] = field(default_factory=list)
    error: str | None = None


async def create_random_persona(
    config: dict, schwartz_config: dict, max_attempts: int = 2
) -> tuple[Persona | None, str]:
    """Generate a random persona with Schwartz values shown through life circumstances."""
    age = random.choice(config["personas"]["age_ranges"])
    prof = random.choice(config["personas"]["professions"])
    cult = random.choice(config["personas"]["cultures"])
    num_values = random.choice([1, 2])
    vals = random.sample(config["personas"]["schwartz_values"], num_values)

    # Build rich value context from the Schwartz elaborations
    value_context = build_value_context(vals, schwartz_config)

    prompt = persona_generation_prompt.render(
        age=age,
        profession=prof,
        culture=cult,
        values=vals,
        value_context=value_context,
        banned_terms=SCHWARTZ_BANNED_TERMS,
    )

    first_person_pattern = re.compile(r"(?i)\b(i|my|me)\b")
    last_persona: Persona | None = None

    for _ in range(max_attempts):
        raw_json = await generate_completion(
            prompt, response_format=PERSONA_RESPONSE_FORMAT
        )
        if not raw_json:
            continue

        data = json.loads(raw_json)
        data["core_values"] = vals  # Ensure correct values
        persona = Persona(**data)
        last_persona = persona

        # Only validate banned terms and first-person usage
        if BANNED_PATTERN.search(persona.bio) or first_person_pattern.search(
            persona.bio
        ):
            continue
        return persona, prompt

    return last_persona, prompt


async def generate_journal_entry(
    persona: Persona,
    config: dict,
    date_str: str,
    previous_entries: list[JournalEntry] | None = None,
    max_attempts: int = 2,
) -> tuple[tuple[JournalEntry, str, str, str] | None, str]:
    """Generate a journal entry for a persona on a given date.

    Returns:
        Tuple of ((entry, tone, verbosity, reflection_mode) or None, prompt used)
    """
    tone = random.choice(config["journal_entries"]["tones"])
    verbosity = random.choice(config["journal_entries"]["verbosity"])
    reflection_mode = random.choice(config["journal_entries"]["reflection_mode"])
    min_words, max_words, max_paragraphs = _verbosity_targets(verbosity)

    # Format previous entries for the prompt
    prev_entries_data = None
    if previous_entries:
        prev_entries_data = [
            {"date": e.date, "content": e.content} for e in previous_entries
        ]

    prompt = journal_entry_prompt.render(
        name=persona.name,
        age=persona.age,
        profession=persona.profession,
        culture=persona.culture,
        bio=persona.bio,
        date=date_str,
        tone=tone,
        verbosity=verbosity,
        min_words=min_words,
        max_words=max_words,
        max_paragraphs=max_paragraphs,
        reflection_mode=reflection_mode,
        previous_entries=prev_entries_data,
    )

    last_entry: JournalEntry | None = None

    for _ in range(max_attempts):
        raw_json = await generate_completion(
            prompt, response_format=JOURNAL_ENTRY_RESPONSE_FORMAT
        )
        if not raw_json:
            continue

        entry = JournalEntry(**json.loads(raw_json))
        last_entry = entry

        # Only validate banned terms (prevent label leakage)
        if not BANNED_PATTERN.search(entry.content):
            return (entry, tone, verbosity, reflection_mode), prompt

    if last_entry:
        return (last_entry, tone, verbosity, reflection_mode), prompt
    return None, prompt


async def generate_conversational_entry(
    persona: Persona,
    config: dict,
    date_str: str,
    previous_entries: list[ConversationalEntry] | None = None,
) -> tuple[ConversationalEntry | None, str, str | None, str | None]:
    """Generate entry, decide on nudge, optionally generate response.

    Returns:
        Tuple of (ConversationalEntry or None, entry_prompt, nudge_prompt, response_prompt)
    """
    # Step 1: Generate initial entry
    prev_journal_entries = [e.initial_entry for e in (previous_entries or [])]
    entry_result, entry_prompt = await generate_journal_entry(
        persona, config, date_str, previous_entries=prev_journal_entries
    )

    if not entry_result:
        return None, entry_prompt, None, None

    entry, tone, verbosity, reflection_mode = entry_result

    # Step 2: Anti-annoyance check (pure, no LLM call needed)
    if previous_entries and should_suppress_nudge(
        [e.nudge is not None for e in previous_entries]
    ):
        return (
            ConversationalEntry(
                initial_entry=entry,
                nudge=None,
                response=None,
                tone=tone,
                verbosity=verbosity,
                reflection_mode=reflection_mode,
                response_mode=None,
            ),
            entry_prompt,
            None,
            None,
        )

    # Step 3: Decide whether to nudge (LLM-based semantic classification)
    # format_previous_entries strips metadata — only date + content pass through
    prev_data = format_previous_entries(
        [
            {"date": e.initial_entry.date, "content": e.initial_entry.content}
            for e in (previous_entries or [])
        ]
    )

    do_nudge, nudge_category, trigger_reason = await decide_nudge(
        entry_content=entry.content,
        entry_date=entry.date,
        previous_entries=prev_data,
        llm_complete=generate_completion,
    )

    nudge_result = None
    response = None
    nudge_prompt = None
    response_prompt = None
    response_mode = None

    if do_nudge and nudge_category:
        # Step 4: Generate nudge text
        nudge_text, nudge_prompt = await generate_nudge_text(
            entry_content=entry.content,
            entry_date=entry.date,
            category=nudge_category,
            previous_entries=prev_data,
            config=config,
            llm_complete=generate_completion,
        )

        if nudge_text:
            nudge_result = NudgeResult(
                nudge_text=nudge_text,
                nudge_category=nudge_category,
                trigger_reason=trigger_reason or "",
            )

            # Step 5: Decide if persona responds (probabilistic)
            if random.random() < config["nudge"]["response_probability"]:
                (
                    response,
                    response_prompt,
                    response_mode,
                ) = await generate_nudge_response(
                    persona_name=persona.name,
                    persona_age=persona.age,
                    persona_profession=persona.profession,
                    persona_culture=persona.culture,
                    persona_bio=persona.bio,
                    entry_content=entry.content,
                    entry_date=entry.date,
                    nudge_text=nudge_text,
                    config=config,
                    llm_complete=generate_completion,
                )
                if response:
                    nudge_result.was_responded_to = True

    return (
        ConversationalEntry(
            initial_entry=entry,
            nudge=nudge_result,
            response=response,
            tone=tone,
            verbosity=verbosity,
            reflection_mode=reflection_mode,
            response_mode=response_mode,
        ),
        entry_prompt,
        nudge_prompt,
        response_prompt,
    )


async def generate_conversational_pipeline(
    persona_id: int,
    config: dict,
    schwartz_config: dict,
    num_entries: int = 3,
    start_date: str = "2023-10-27",
) -> ConversationalPipelineResult:
    """Generate one persona and all their conversational journal entries."""
    entry_prompts: list[str] = []
    nudge_prompts: list[str] = []
    response_prompts: list[str] = []
    entries: list[ConversationalEntry] = []

    # 1. Generate persona
    persona, persona_prompt = await create_random_persona(config, schwartz_config)

    if not persona:
        return ConversationalPipelineResult(
            persona_id=persona_id,
            persona=None,
            entries=[],
            persona_prompt=persona_prompt,
            entry_prompts=[],
            error="Failed to generate persona",
        )

    # 2. Generate conversational entries sequentially
    dates = generate_date_sequence(start_date, num_entries)

    for date_str in dates:
        (
            conv_entry,
            entry_prompt,
            nudge_prompt,
            response_prompt,
        ) = await generate_conversational_entry(
            persona, config, date_str, previous_entries=entries
        )
        entry_prompts.append(entry_prompt)
        if nudge_prompt:
            nudge_prompts.append(nudge_prompt)
        if response_prompt:
            response_prompts.append(response_prompt)

        if conv_entry:
            entries.append(conv_entry)

    return ConversationalPipelineResult(
        persona_id=persona_id,
        persona=persona,
        entries=entries,
        persona_prompt=persona_prompt,
        entry_prompts=entry_prompts,
        nudge_prompts=nudge_prompts,
        response_prompts=response_prompts,
        error=None,
    )


async def run_parallel_conversational_personas(
    num_personas: int,
    config: dict,
    schwartz_config: dict,
    min_entries: int = 3,
    max_entries: int = 10,
    start_date: str = "2023-10-27",
) -> list[ConversationalPipelineResult | Exception]:
    """Run multiple conversational persona pipelines in parallel.

    Args:
        num_personas: Number of personas to generate in parallel
        config: Main configuration dict
        schwartz_config: Schwartz values elaboration config
        min_entries: Minimum journal entries per persona
        max_entries: Maximum journal entries per persona
        start_date: Starting date for journal entries

    Returns:
        List of ConversationalPipelineResult or Exception, in persona order
    """
    # Each persona gets a random number of entries for training diversity
    tasks = [
        generate_conversational_pipeline(
            i + 1,
            config,
            schwartz_config,
            num_entries=random.randint(min_entries, max_entries),
            start_date=start_date,
        )
        for i in range(num_personas)
    ]

    results = await asyncio.gather(*tasks, return_exceptions=True)
    return list(results)

## Output Logging System

In [ ]:
def get_log_dir() -> Path:
    """Create and return a timestamped log directory."""
    base_dir = Path("logs/synthetic_data")
    if not base_dir.exists():
        base_dir = Path("../logs/synthetic_data")
    base_dir.mkdir(parents=True, exist_ok=True)

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_dir = base_dir / timestamp
    log_dir.mkdir(exist_ok=True)
    return log_dir


def write_config_log(
    log_dir: Path, config: dict, num_personas: int, min_entries: int, max_entries: int
) -> None:
    """Write config.md with run parameters."""
    content = f"""# Run Configuration

**Timestamp**: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
**Notebook**: journal_nudge.ipynb

## Persona Generation
- Num personas: {num_personas}
- Entries per persona: {min_entries}-{max_entries} (variable)

## Nudge Settings
- Decision method: LLM-based classification (prompts/nudge_decision.yaml)
- Response probability: {config["nudge"]["response_probability"]}

## Model Settings
- Model: {MODEL_NAME}
- Reasoning effort: {DEFAULT_REASONING_EFFORT}
"""
    (log_dir / "config.md").write_text(content)


def write_persona_log(log_dir: Path, result: ConversationalPipelineResult) -> None:
    """Write persona_XXX.md with all entries, nudges, and summary statistics."""
    if not result.persona:
        return

    p = result.persona
    lines = [
        f"# Persona {result.persona_id:03d}: {p.name}",
        "",
        "## Profile",
        f"- Age: {p.age}",
        f"- Profession: {p.profession}",
        f"- Culture: {p.culture}",
        f"- Core Values: {', '.join(p.core_values)}",
        f"- Bio: {p.bio}",
        "",
        "---",
    ]

    for i, entry in enumerate(result.entries, 1):
        lines.extend(
            [
                "",
                f"## Entry {i} - {entry.initial_entry.date}",
                "",
                "### Initial Entry",
                f"**Tone**: {entry.tone} | **Verbosity**: {entry.verbosity} | **Reflection Mode**: {entry.reflection_mode}",
                "",
                entry.initial_entry.content,
            ]
        )

        if entry.nudge:
            lines.extend(
                [
                    "",
                    f"### Nudge ({entry.nudge.nudge_category.replace('_', ' ').title()})",
                    f"**Trigger**: {entry.nudge.trigger_reason}",
                    "",
                    f'"{entry.nudge.nudge_text}"',
                ]
            )

            if entry.response:
                lines.extend(
                    [
                        "",
                        "### Response",
                        f"**Mode**: {entry.response_mode or 'Unknown'}",
                        "",
                        entry.response.content,
                    ]
                )
            else:
                lines.append("\n*(No response - persona did not reply to nudge)*")
        else:
            lines.append("\n*(No nudge for this entry)*")

        lines.extend(["", "---"])

    # Summary Statistics
    nudge_count = sum(1 for e in result.entries if e.nudge)
    response_count = sum(1 for e in result.entries if e.nudge and e.response)

    # Count nudge categories
    category_counts = {"clarification": 0, "elaboration": 0, "tension_surfacing": 0}
    for e in result.entries:
        if e.nudge:
            category_counts[e.nudge.nudge_category] += 1

    # Count response modes
    mode_counts = {
        "Answering directly": 0,
        "Deflecting/redirecting": 0,
        "Revealing deeper thought": 0,
    }
    for e in result.entries:
        if e.response and e.response_mode:
            if e.response_mode in mode_counts:
                mode_counts[e.response_mode] += 1

    lines.extend(
        [
            "",
            "## Summary Statistics",
            "",
            "| Metric | Value |",
            "|--------|-------|",
            f"| Total Entries | {len(result.entries)} |",
            f"| Nudges Generated | {nudge_count} |",
            f"| Responses Given | {response_count} |",
            f"| Nudge Categories | clarification ({category_counts['clarification']}), elaboration ({category_counts['elaboration']}), tension_surfacing ({category_counts['tension_surfacing']}) |",
            f"| Response Modes | Answering directly ({mode_counts['Answering directly']}), Deflecting ({mode_counts['Deflecting/redirecting']}), Revealing deeper ({mode_counts['Revealing deeper thought']}) |",
        ]
    )

    (log_dir / f"persona_{result.persona_id:03d}.md").write_text("\n".join(lines))


def write_prompts_log(
    log_dir: Path, results: list[ConversationalPipelineResult]
) -> None:
    """Write prompts.md with all LLM prompts."""
    lines = ["# Prompts Log", ""]

    for result in results:
        if isinstance(result, Exception) or not result.persona:
            continue

        lines.extend(
            [
                f"## Persona {result.persona_id:03d}: {result.persona.name}",
                "",
                "### Persona Generation Prompt",
                "```",
                result.persona_prompt,
                "```",
                "",
            ]
        )

        for i, prompt in enumerate(result.entry_prompts, 1):
            lines.extend(
                [
                    f"### Entry {i} - Initial Entry Prompt",
                    "```",
                    prompt,
                    "```",
                    "",
                ]
            )

        if result.nudge_prompts:
            for i, prompt in enumerate(result.nudge_prompts, 1):
                lines.extend(
                    [
                        f"### Nudge Prompt {i}",
                        "```",
                        prompt,
                        "```",
                        "",
                    ]
                )

        if result.response_prompts:
            for i, prompt in enumerate(result.response_prompts, 1):
                lines.extend(
                    [
                        f"### Response Prompt {i}",
                        "```",
                        prompt,
                        "```",
                        "",
                    ]
                )

        lines.append("---\n")

    (log_dir / "prompts.md").write_text("\n".join(lines))


def save_run_logs(
    results: list[ConversationalPipelineResult | Exception],
    config: dict,
    num_personas: int,
    min_entries: int,
    max_entries: int,
) -> Path:
    """Save all logs for a run.

    Returns:
        Path to the log directory
    """
    log_dir = get_log_dir()

    # Filter successful results
    successful = [
        r for r in results if isinstance(r, ConversationalPipelineResult) and r.persona
    ]

    write_config_log(log_dir, config, num_personas, min_entries, max_entries)

    for result in successful:
        write_persona_log(log_dir, result)

    write_prompts_log(log_dir, successful)

    print(f"Logs saved to: {log_dir}")
    return log_dir

## Display Functions

In [13]:
def display_conversational_results(
    result: ConversationalPipelineResult | Exception,
) -> None:
    """Display all outputs for one persona."""
    if isinstance(result, Exception):
        print(f"\n{'=' * 80}")
        print(f"PERSONA FAILED WITH EXCEPTION:")
        print(f"{'=' * 80}")
        print(f"{type(result).__name__}: {result}")
        print(f"{'=' * 80}\n")
        return

    print(f"\n{'=' * 80}")
    print(f"PERSONA {result.persona_id}")
    print(f"{'=' * 80}")

    if result.error:
        print(f"\nError: {result.error}")
        return

    # Persona details
    p = result.persona
    print(f"\n## Generated Persona: {p.name}")
    print(f"Age: {p.age} | Profession: {p.profession} | Culture: {p.culture}")
    print(f"Values: {', '.join(p.core_values)}")
    print(f"Bio: {p.bio}")

    # Entries with nudges
    for i, entry in enumerate(result.entries, 1):
        print(f"\n{'─' * 40}")
        print(f"### Entry {i}: {entry.initial_entry.date}")
        print(
            f"Tone: {entry.tone} | Verbosity: {entry.verbosity} | Mode: {entry.reflection_mode}"
        )
        print(f"\n**Initial Entry:**")
        print(entry.initial_entry.content)

        if entry.nudge:
            print(f"\n**Nudge ({entry.nudge.nudge_category}):**")
            print(f"Trigger: {entry.nudge.trigger_reason}")
            print(f'"{entry.nudge.nudge_text}"')

            if entry.response:
                print(f"\n**Response:**")
                print(entry.response.content)
            else:
                print("\n*(No response)*")
        else:
            print("\n*(No nudge)*")

    # Summary stats
    nudge_count = sum(1 for e in result.entries if e.nudge)
    response_count = sum(1 for e in result.entries if e.nudge and e.response)
    print(f"\n{'─' * 40}")
    print(f"### Summary for {p.name}")
    print(f"Total entries: {len(result.entries)}")
    print(f"Nudges given: {nudge_count}")
    print(f"Responses received: {response_count}")

# Execution Loop

In [ ]:
# Configuration
NUM_PERSONAS = 3
MIN_ENTRIES = 3
MAX_ENTRIES = 10
START_DATE = "2025-10-25"

print(f"Generating {NUM_PERSONAS} personas with conversational journaling...")
print(f"Each persona will have {MIN_ENTRIES}-{MAX_ENTRIES} entries with potential nudges.")
print(f"Model: {MODEL_NAME} | Reasoning: {DEFAULT_REASONING_EFFORT}")
print(f"Start date: {START_DATE}")
print(f"Nudge decision: LLM-based classification (prompts/nudge_decision.yaml)")
print(f"Response probability: {config['nudge']['response_probability']}\n")

# Run all personas in parallel
results = await run_parallel_conversational_personas(
    num_personas=NUM_PERSONAS,
    config=config,
    schwartz_config=schwartz_config,
    min_entries=MIN_ENTRIES,
    max_entries=MAX_ENTRIES,
    start_date=START_DATE,
)

# Display results
for result in results:
    display_conversational_results(result)

# Save logs
successful_results = [
    r for r in results if isinstance(r, ConversationalPipelineResult) and r.persona
]
log_dir = save_run_logs(results, config, NUM_PERSONAS, MIN_ENTRIES, MAX_ENTRIES)

# Final summary
print(f"\n{'=' * 80}")
print(f"FINAL SUMMARY")
print(f"{'=' * 80}")
print(f"Successfully generated: {len(successful_results)}/{NUM_PERSONAS} personas")

total_entries = sum(len(r.entries) for r in successful_results)
entry_counts = [len(r.entries) for r in successful_results]
total_nudges = sum(sum(1 for e in r.entries if e.nudge) for r in successful_results)
total_responses = sum(
    sum(1 for e in r.entries if e.nudge and e.response) for r in successful_results
)

print(f"Total entries: {total_entries}")
if entry_counts:
    print(f"Entries per persona: min={min(entry_counts)}, max={max(entry_counts)}, avg={sum(entry_counts)/len(entry_counts):.1f}")
print(f"Total nudges given: {total_nudges}")
print(f"Total responses: {total_responses}")
if total_nudges > 0:
    print(f"Response rate: {total_responses / total_nudges:.1%}")
print(f"\nLogs saved to: {log_dir}")